In [ ]:
%load_ext jupyter_black

In [ ]:
import pandas as pd
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz

from utils import plot_line, plot_quantiles

In [ ]:
DATA_DIR = "../trades/"
LOG_DIR = "../logs/"
PROCESSED_DIR = "./trades/"

DATA_FILE = "hatguy"

BASE = "WIF"
QUOTE = "USDC"
PRICE_PER_TICK = 0.0001
UNITS_PER_LOT = 0.01

### Preprocessing

In [ ]:
data_path = f"{DATA_DIR}{DATA_FILE}.csv"
processed_data_path = f"{PROCESSED_DIR}{DATA_FILE}-analysis.csv"

df = pd.read_csv(data_path)

# df

In [ ]:
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
df["timestamp"] = df["timestamp"].dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
df = df.sort_values(by="timestamp")

df["local_timestamp"] = pd.to_datetime(df["local_timestamp"], unit="ms")
df["local_timestamp"] = (
    df["local_timestamp"].dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
)
df = df.sort_values(by="timestamp")

df["price"] = df["price"] * PRICE_PER_TICK
df["size"] = df["size"] * UNITS_PER_LOT


df.head(5)

In [ ]:
df.to_csv(processed_data_path, index=False)

### Overview

In [ ]:
def calculate_pnl(df):
    base_inventory = 0
    quote_inventory = 0

    pnl_series = pd.Series(index=df["timestamp"])

    for index, row in df.iterrows():
        base_value = row["size"]
        quote_value = row["size"] * row["price"]

        if row["side"] == "buy":
            base_inventory += base_value
            quote_inventory -= quote_value
        elif row["side"] == "sell":
            base_inventory -= base_value
            quote_inventory += quote_value

        # assume we're only quoting USDC pairs
        portfolio_value = base_inventory * row["price"] + quote_inventory * 1
        pnl_series.loc[row["timestamp"]] = portfolio_value

    pnl_series = pnl_series.dropna()
    return pnl_series

In [ ]:
df["inventory_change"] = df.apply(
    lambda row: row["size"] if row["side"] == "buy" else -row["size"], axis=1
)
df["cumulative_inventory"] = df["inventory_change"].cumsum()
pnl_series = calculate_pnl(df)
flat_pnl_series = pd.Series(0, index=pnl_series.index)
inventory_series = df.set_index("timestamp")["cumulative_inventory"]
flat_inventory_series = pd.Series(0, index=inventory_series.index)
time_diffs = df["timestamp"].diff().dt.total_seconds()
time_diffs = time_diffs.dropna()

In [ ]:
duration = df["timestamp"].max() - df["timestamp"].min()
hours = duration.components.hours
minutes = duration.components.minutes

trade_volume = df["size"] * df["price"]
quote_volume = trade_volume.sum()
base_volume = df["size"].sum()

In [ ]:
print(f"Made {len(df)} trades over {hours}h {minutes}m")
print(f"Total volume ({QUOTE}): {quote_volume:.2f}")
print(f"Total volume ({BASE}): {base_volume}")
plot_line(
    df["timestamp"], [pnl_series, flat_pnl_series], ["PnL", "Break Even"], "Time", "PnL"
)
plot_line(
    df["timestamp"],
    [inventory_series, flat_inventory_series],
    ["Inventory", "Flat"],
    "Time",
    "Inventory",
)
plot_quantiles(time_diffs, bins=50, name="Time between Trades (s)")
plot_quantiles(df["size"], bins=50, name="Size")

In [ ]:
df.tail(5)